In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

In [8]:
dataTrain = pd.read_csv('/content/drive/MyDrive/Sentiment Analysis Project final/Tweets_train.csv')
dataValidate = pd.read_csv('/content/drive/MyDrive/Sentiment Analysis Project final/Tweets_dev.csv')

In [10]:
print(dataTrain.shape)
print(dataValidate.shape)

(11858, 3)
(1318, 3)


In [11]:
print(dataTrain.head())

             tweet_id                                               text  \
0  569179849518161920                    @united you're good. Thank you!   
1  569835751275433984  @AmericanAir way to ruin a vacation, my brothe...   
2  568588936852799488  @JetBlue yes thankfully! Catering just got her...   
3  569525116725567491  @USAirways The automated message isn't helpful...   
4  568807823187976193  @JetBlue I'm #MakingLoveOutofNothingAtAll on m...   

  airline_sentiment  
0          positive  
1          negative  
2          positive  
3          negative  
4          positive  


In [12]:
print(dataValidate.head())

             tweet_id                                               text  \
0  568107472260624384  @southwestair Great job celebrating #MardiGras...   
1  568215698524246016  @southwestair thanks for taking it up a notch!...   
2  567842466851905536  @USAirways Being put back on hold for what has...   
3  568834824410148864  @united Thank you for your offer! All sorted o...   
4  569590527349252096  @JetBlue wondering if it's possible for my col...   

  airline_sentiment  
0          positive  
1          positive  
2          negative  
3          positive  
4           neutral  


In [13]:
corpus_train=[]
for x in range(0, len (dataTrain)):
    dataClean = re.sub('[^a-zA-Z]',' ',dataTrain['text'][x])
    dataClean = dataClean.lower()
    dataClean = dataClean.split()
    dataClean = [ps.stem(word) for word in dataClean if not word in set(all_stopwords)]
    dataClean=' '.join(dataClean)
    corpus_train.append(dataClean)
corpus_train


['unit good thank',
 'americanair way ruin vacat brother call night multipl place line get dead air call back',
 'jetblu ye thank cater got load frustrat suppos',
 'usairway autom messag help imposs speak human right desper need luggag',
 'jetblu makingloveoutofnothingatal brandloveaffair lax http co kdhruf sw',
 'americanair download app iphon notic drink coupon noth display get readi fli',
 'unit post link articl verifi initi complaint comment say thing harri',
 'jetblu not sure anyth help great',
 'americanair baggag lost flight cancel flightl get accommod not even anoth airlin least one free hotel',
 'usairway charlott staff need prayer disappoint respons mishap lotttttttt piss peopl',
 'usairway suggest failur make huge donat uso charlott nc provid great custom servic today unlik',
 'americanair pleas explain cost almost price full roundtrip fair chang departur time flight',
 'southwestair plane work luxuri paid go away keep money need biz model',
 'delet account jetblu',
 'virgin

In [14]:
corpus_validate=[]
for i in range(0, len (dataValidate)):
    dataClean = re.sub('[^a-zA-Z]',' ',dataValidate['text'][i])
    dataClean = dataClean.lower()
    dataClean = dataClean.split()
    dataClean = [ps.stem(word) for word in dataClean if not word in set(all_stopwords)]
    dataClean=' '.join(dataClean)
    corpus_validate.append(dataClean)
corpus_validate

['southwestair great job celebr mardigra industri anoth reason nut http co wbzorrn c',
 'southwestair thank take notch leinenkugel craftbeer goodflight norfolk http co tgsljjn g',
 'usairway put back hold hour complet unaccept',
 'unit thank offer sort',
 'jetblu wonder possibl colleagu get earlier flight lax gt jfk tomorrow help',
 'unit common keep paper work readi delay flight meet choosechicago',
 'unit keep get kick error messag tri purchas flight http co p n w jfp tri least x',
 'jetblu flight fli way bostonlogan mind take ocean approach sleep',
 'jetblu left peopl hot plane ventil hour half bu jfk',
 'unit thank sent',
 'usairway askpayp mark complet scare go lose reserv money go refund',
 'usairway pleas dm flight delay go clt forc miss connect lga pm',
 'usairway know deal weather phlairport hot mess pl send reinforc',
 'usairway jfk baggag offic open help book us cancel flightl flight dca shame',
 'jetblu home guy recov laugh extra day barbado open cuba soon',
 'usairway bett

In [43]:
cv_Train = CountVectorizer(max_features=500)
x= cv_Train.fit_transform(corpus_train).toarray()
y = list(dataTrain.iloc[:,-1].values)

cv_Validate = CountVectorizer(max_features=500)
x1=cv_Validate.fit_transform(corpus_validate).toarray()
y1= list(dataValidate.iloc[:,-1].values)

In [44]:
import pickle

bow_path = 'DataTrain.pkl'
pickle.dump(cv_Train, open(bow_path,"wb"))

bow_path1 = 'DataValidate.pkl'
pickle.dump(cv_Validate, open(bow_path1,"wb"))

In [45]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x,y)

import joblib
joblib.dump(classifier,"NaiveBayesModel")

['NaiveBayesModel']

In [46]:
y_prediction = classifier.predict(x1)

In [47]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y1,y_prediction)
print(cm)

print(accuracy_score(y1,y_prediction))

[[536 109 181]
 [113  80  86]
 [ 83  55  75]]
0.5242792109256449


In [48]:
dataTest = pd.read_csv('/content/drive/MyDrive/Sentiment Analysis Project final/New_Tweets_test-1.csv')
print(dataTest.shape)

(1464, 4)


In [49]:
corpus_Test=[]
for i in range(0, len (dataTest)):
    dataClean = re.sub('[^a-zA-Z]',' ',dataTest['text'][i])
    dataClean = dataClean.lower()
    dataClean = dataClean.split()
    dataClean = [ps.stem(word) for word in dataClean if not word in set(all_stopwords)]
    dataClean=' '.join(dataClean)
    corpus_Test.append(dataClean)
print(corpus_Test)

['americanair need refund', 'usairway cancel flightlat delay caus miss connect flight anoth night not home thank pathet', 'jetblu thank much wait fli guy', 'unit never frustrat convers unit speak spanish courier compani bag', 'usairway worst hold time crazi agent horribl account usairsuck', 'americanair pleasur next aa flight wednesday milan itali midoexhibit see', 'unit frankli wors custom servic ever problem happen deal defin compani never unit', 'southwestair need get act togeth new morn plane malfunct yet delay time', 'americanair cancel flight flight left help find hotel stay slept airport night', 'jetblu flight bo tampa takeoff min late flight', 'unit start xweekli b flight neworlean cancun may sep avgeek', 'jetblu airway short interest januari jblu sleekmoney http co znsujp bv', 'americanair might not urgent realli', 'usairway hold hr min ridicul', 'southwestair statu flight stl au', 'usairway guy switch inform chath', 'jetblu tell paid amp confirm flight cost cash airport hour 

In [50]:
import pickle
cv_Train = 'DataTrain.pkl'
cv_File = pickle.load(open(cv_Train,"rb"))

In [51]:
X_fresh = cv_File.transform(corpus_Test).toarray()
print(X_fresh.shape)

(1464, 500)


In [54]:

classifier = joblib.load("NaiveBayesModel")
count_true = 0
y_predictionTest = classifier.predict(X_fresh)
y_predictionTest

array(['neutral', 'negative', 'positive', ..., 'positive', 'neutral',
       'negative'], dtype='<U8')

In [55]:
y2= list(dataTest.iloc[:,-1].values)

In [56]:
print(accuracy_score(y2,y_predictionTest))

0.5478142076502732
